In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import requests_cache
from matplotlib.transforms import Bbox

from moonbox import now, get_oneday, parse_oneday, draw_moon

In [ ]:
session = requests_cache.CachedSession("cache")

year = now().year

dates = (
    pl.select(pl.date_range(pl.date(year, 1, 1), pl.date(year, 12, 31)))
    .to_series()
    .to_list()
)

data = [
    {"date": date} | parse_oneday(get_oneday(date.isoformat(), session=session))
    for date in dates
]

In [ ]:
def chunk(lst, pred, min_len=2):
    out = []
    chunk = None
    for elt in lst:
        if chunk is None:
            if pred(elt):
                chunk = [elt]
            else:
                pass
        elif pred(elt) and len(chunk) + 1 > min_len:
            out.append(chunk)
            chunk = [elt]
        else:
            chunk.append(elt)

    return out

In [ ]:
# group into lunar months
lunar_months = chunk(data, lambda x: x["phase"] == "New Moon")

n_months = len(lunar_months)
longest_month = max([len(x) for x in lunar_months])

scale = 1.0
fig, axs = plt.subplots(
    n_months,
    longest_month,
    figsize=(scale * longest_month, scale * n_months),
    layout="compressed",
)

# set everything to not visible, to deal with ragged rows
for row in range(n_months):
    for col in range(longest_month):
        axs[row, col].set_visible(False)

solar_month = None

for row, month in enumerate(lunar_months):
    for col, datum in enumerate(month):
        if col == 0 or datum["date"].month != solar_month:
            label = datum["date"].strftime("%b %-d")
            solar_month = datum["date"].month
        else:
            label = datum["date"].strftime("%-d")

        f = datum["illumination"] / 100
        direction = datum["phase"].split()[0].lower()

        ax = axs[row, col]
        ax.set_visible(True)
        draw_moon(ax, f=f, direction=direction, dark="0.2")
        ax.text(
            0.875,  # x-position of end of text
            1.0,  # y-position of top of text
            label,
            horizontalalignment="right",
            verticalalignment="bottom",
            transform=ax.transAxes,
            color="white",
        )

fig.suptitle(f"{year} Lunar Calendar", fontsize=60, color="white")
fig.set_facecolor("black")

In [ ]:
# use this code to see how big the figure is, and then trim (or expand) to some other size

h0 = fig.get_figheight()
w0 = fig.get_figwidth()

x = 1.5
h = 20.0 - 2.0 * x
w = 30.0 - 3.0 * x

print(f"w0={w0} h0={h0}")
print(f"w={w} h={h}")

dw = (w0 - w) / 2
dh = (h0 - h) / 2

bb = Bbox([[dw, dh], [w0 - dw, h0 - dh]])
fig.savefig("../output/lunar_calendar_trimmed.png", bbox_inches=bb, dpi=300)
fig.savefig("../output/lunar_calendar_trimmed.pdf", bbox_inches=bb)